FastAPI 提供了简单易用，但功能强大的依赖注入系统。

这个依赖系统设计的简单易用，可以让开发人员轻松地把组件集成至 FastAP

什么是「依赖注入」

编程中的「依赖注入」是声明代码（本文中为路径操作函数 ）运行所需的，或要使用的「依赖」的一种方式。

然后，由系统（本文中为 FastAPI）负责执行任意需要的逻辑，为代码提供这些依赖（「注入」依赖项）。

依赖注入常用于以下场景：

共享业务逻辑（复用相同的代码逻辑）
共享数据库连接
实现安全、验证、角色权限
等……
上述场景均可以使用依赖注入，将代码重复最小化。I。

创建依赖项
首先，要关注的是依赖项。

依赖项就是一个函数，且可以使用与路径操作函数相同的

声明依赖项
与在路径操作函数参数中使用 Body、Query 的方式相同，声明依赖项需要使用 Depends 和一个新的参数：
参数：

In [1]:
import uvicorn
from fastapi import Depends, FastAPI
app = FastAPI()

async def common_parameters(
    q: str | None = None, skip: int = 0, limit: int = 100
):
    return {"q": q, "skip": skip, "limit": limit}


@app.get("/items/")
async def read_items(commons: dict = Depends(common_parameters)):
    return commons


@app.get("/users/")
async def read_users(commons: dict = Depends(common_parameters)):
    return commons

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [22264]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:53912 - "GET /items/?q=%E5%BC%A0%E4%B8%89 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53925 - "GET /users/?q=%E6%9D%8E%E5%9B%9B HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [22264]


依赖注入无非是与路径操作函数一样的函数罢了。

大功告成。

只用了2 行代码。

依赖项函数的形式和结构与路径操作函数一样。

因此，可以把依赖项当作没有「装饰器」（即，没有 @app.get("/some-path") ）的路径操作函数。

依赖项可以返回各种内容。

本例中的依赖项预期接收如下参数：

类型为 str 的可选查询参数 q
类型为 int 的可选查询参数 skip，默认值是 0
类型为 int 的可选查询参数 limit，默认值是 100
然后，依赖项函数返回包含这些值的 dict。

In [ ]:
url = 'http://127.0.0.1:8009/items/?q=张三' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"q":"张三","skip":0,"limit":100}'

In [ ]:
url = 'http://127.0.0.1:8009/users/?q=李四' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"q":"李四","skip":0,"limit":100}'

虽然，在路径操作函数的参数中使用 Depends 的方式与 Body、Query 相同，但 Depends 的工作方式略有不同。

这里只能传给 Depends 一个参数。

且该参数必须是可调用对象，比如函数。

该函数接收的参数和路径操作函数的参接收到新的请求时，FastAPI 执行如下操作：

用正确的参数调用依赖项函数（「可依赖项」）
获取函数返回的结果
把函数返回的结果赋值给路径操作函

这样，只编写一次代码，FastAPI 就可以为多个路径操作共享这段代码 。数的参数数一样。

：

虽然，层级式依赖注入系统的定义与使用十分简单，但它却非常强大。

比如，可以定义依赖其他依赖项的依赖项。

最后，依赖项层级树构建后，依赖注入系统会处理所有依赖项及其子依赖项，并为每一步操作提供（注入）结果。